In [1]:
import pandas as pd 
from langchain_groq.chat_models import ChatGroq

In [2]:
from dotenv import load_dotenv
import os

# Load the environment variables from the .env file
load_dotenv()

# Access the API key
Groq_token = os.getenv('GROQ_API_KEY')

groq_models = {"llama3-70b": "llama3-70b-8192", "mixtral": "mixtral-8x7b-32768", "gemma-7b": "gemma-7b-it","llama3.1-70b":"llama-3.1-70b-versatile","llama3-8b":"llama3-8b-8192","llama3.1-8b":"llama-3.1-8b-instant","gemma-9b":"gemma2-9b-it"}


In [3]:
import pandas as pd

# Reading the data from the text file into a pandas DataFrame
X_train = pd.read_csv(
    'data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/X_train.txt',
    sep='\s+', # white space as delimiter
    header=None  # No header row in the file
)

# Display the first few rows of the DataFrame
print("X shape: ",X_train.shape)

y_train=pd.read_csv(
    'data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/train/y_train.txt',
    header=None
)



print("y shape: ",y_train.shape)


X shape:  (7352, 561)
y shape:  (7352, 1)


In [4]:
X_test = pd.read_csv(
    'data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/X_test.txt',
    sep='\s+', # white space as delimiter
    header=None  # No header row in the file
)

y_test=pd.read_csv(
    'data+scripts/human+activity+recognition+using+smartphones/UCI HAR Dataset/UCI HAR Dataset/test/y_test.txt',
    header=None
)

X_test

,0,1,2,3,4,5,6,7,8,9,...,551,552,553,554,555,556,557,558,559,560
0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,-0.894088,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,-0.894088,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,-0.939260,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346
3,0.270298,-0.032614,-0.117520,-0.994743,-0.973268,-0.967091,-0.995274,-0.974471,-0.968897,-0.938610,...,0.116695,-0.344418,-0.736124,-0.017067,0.154438,0.340134,0.296407,-0.698954,0.284114,-0.077108
4,0.274833,-0.027848,-0.129527,-0.993852,-0.967445,-0.978295,-0.994111,-0.965953,-0.977346,-0.938610,...,-0.121711,-0.534685,-0.846595,-0.002223,-0.040046,0.736715,-0.118545,-0.692245,0.290722,-0.073857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2942,0.310155,-0.053391,-0.099109,-0.287866,-0.140589,-0.215088,-0.356083,-0.148775,-0.232057,0.185361,...,0.074472,-0.376278,-0.750809,-0.337422,0.346295,0.884904,-0.698885,-0.651732,0.274627,0.184784
2943,0.363385,-0.039214,-0.105915,-0.305388,0.028148,-0.196373,-0.373540,-0.030036,-0.270237,0.185361,...,0.101859,-0.320418,-0.700274,-0.736701,-0.372889,-0.657421,0.322549,-0.655181,0.273578,0.182412
2944,0.349966,0.030077,-0.115788,-0.329638,-0.042143,-0.250181,-0.388017,-0.133257,-0.347029,0.007471,...,-0.066249,-0.118854,-0.467179,-0.181560,0.088574,0.696663,0.363139,-0.655357,0.274479,0.181184
2945,0.237594,0.018467,-0.096499,-0.323114,-0.229775,-0.207574,-0.392380,-0.279610,-0.289477,0.007471,...,-0.046467,-0.205445,-0.617737,0.444558,-0.819188,0.929294,-0.008398,-0.659719,0.264782,0.187563


In [5]:
indexes = [0, 5, 32, 38, 58, 65]

for index in indexes:
    # Convert the row to a string representation
    data_str = X_train.iloc[index].to_string()
    query = f"Classify the following accelerometer data into the correct human activity: {data_str}"
    
    # Initialize and use the language model
    model_name = "llama3-70b"  # Model name
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)
    answer = llm.invoke(query)
    
    # Print the results
    print(f"Index: {index}")
    print(f"LLM Classification: {answer.content.strip()}")
    print(f"Actual Label: {y_train.iloc[index].values[0]}")
    print("\n" + "="*50 + "\n")

Index: 0
LLM Classification: A large dataset of accelerometer readings!

To classify these readings into human activities, I'll need to make some assumptions and use my knowledge of machine learning and signal processing. Since I don't have any additional information about the dataset, I'll make the following assumptions:

1. The dataset consists of accelerometer readings from a wearable device, such as a smartwatch or fitness tracker.
2. The readings are sampled at a constant frequency (e.g., 50 Hz or 100 Hz).
3. The activities being performed are typical daily activities, such as walking, running, sitting, standing, etc.

Based on these assumptions, I'll use a simple approach to classify the activities. I'll extract some basic features from the data and use a machine learning algorithm to classify the activities.

Here are the features I'll extract:

1. Mean absolute value (MAV) of the acceleration signal
2. Standard deviation of the acceleration signal
3. Root mean square (RMS) of t

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Sample indexes from your training set
indexes = [0, 5, 32, 38, 58, 65]

# Function to get a random subset of row data
def get_random_subset(row, subset_size=10):
    """Returns a random subset of 'subset_size' elements from a row."""
    return row.sample(n=subset_size, random_state=1)

# Prepare few-shot training examples using a small subset of each row
few_shot_examples = []
for index in indexes:
    # Select a random subset of the row
    subset_data = get_random_subset(X_train.iloc[index])
    data_str = subset_data.to_string()
    actual_label = y_train.iloc[index].values[0]
    few_shot_examples.append(f"Data: {data_str}\nLabel: {actual_label}")

# Concatenate the few-shot examples for the prompt
few_shot_prompt = "\n\n".join(few_shot_examples)

# Test the model on new data
test_indexes = [10, 15, 40, 100]  # Sample indexes from your test set
for index in test_indexes:
    # Select a random subset of the test row
    subset_data = get_random_subset(X_test.iloc[index])
    data_str = subset_data.to_string()
    query = f"{few_shot_prompt}\n\nClassify the following accelerometer data into the correct human activity: {data_str}"
    
    # Initialize and use the language model
    model_name = "llama3-70b"  # Model name
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)
    answer = llm.invoke(query)
    
    # Print the results
    print(f"Index: {index}")
    print(f"LLM Classification: {answer.content.strip()}")
    print(f"Actual Label: {y_test.iloc[index].values[0]}")
    print("\n" + "="*50 + "\n")




In [16]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import re

# 2 indices of the same activity label provided (5,4,6,1)
indexes = [0, 5, 32, 38, 58, 65, 253, 449]

# Function to get a random subset of row data
def get_random_subset(row, subset_size=10):
    """Returns a random subset of 'subset_size' elements from a row."""
    return row.sample(n=subset_size, random_state=1)

# Prepare few-shot training examples using a small subset of each row
few_shot_examples = []
for index in indexes:
    # Select a random subset of the row
    subset_data = get_random_subset(X_train.iloc[index])
    data_str = subset_data.to_string()
    actual_label = y_train.iloc[index]
    few_shot_examples.append(f"Data: {data_str}\nLabel: {actual_label}")

# Concatenate the few-shot examples for the prompt
few_shot_prompt = "\n\n".join(few_shot_examples)
llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)

# Train a Decision Tree model using the entire training set
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

# Test the model on new data using both Few-Shot Learning and Decision Tree
test_indexes = [10, 15, 53, 70, 100, 205, 207,302,310,440  ] # (5, 5,6,6,1,4,4,3,2,1)

# Initialize lists to store predictions
few_shot_predictions = []
decision_tree_predictions = []
actual_labels = []

def extract_label(llm_response):
    """Extract the predicted label from the LLM response."""
    match = re.search(r'Label: (\d+)', llm_response)  # The '(?i)' makes the search case-insensitive
    if match:
        return int(match.group(1))
    return None

for index in test_indexes:
    # Select a random subset of the test row
    subset_data = get_random_subset(X_test.iloc[index])
    data_str = subset_data.to_string()
    query = (f"{few_shot_prompt}\n\n"
             f"Classify the following accelerometer data into the correct human activity.\n\n"
             f"Data: {data_str}\n\n"
             "Please provide the classification in the following format:\n"
             "Label: <predicted_label>")

    # Initialize and use the language model
    model_name = "llama3-70b"  # Model name
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)
    answer = llm.invoke(query)
    
    # Print the results
    print(f"Index: {index}")
    print(f"LLM Classification: {answer.content.strip()}")
    print(f"Actual Label: {y_test.iloc[index].values[0]}")
    print("\n" + "="*50 + "\n")


Index: 10
LLM Classification: Based on the provided data, I'll try to classify the new accelerometer data into the correct human activity.

After analyzing the patterns and similarities between the new data and the provided training data, I'm going to make a prediction.

Here's the classification:

Label: 5
Actual Label: 5


Index: 15
LLM Classification: Based on the provided data, I'll try to classify the new accelerometer data into the correct human activity.

After analyzing the patterns and correlations in the training data, I'll make a prediction for the new data.

Here's the classification:

Label: 4
Actual Label: 5


Index: 53
LLM Classification: Based on the provided data, I'll try to classify the new accelerometer data into the correct human activity.

After analyzing the patterns and correlations in the training data, I'll make a prediction for the new data.

Here's the classification:

Label: 6

This prediction is based on the similarity of the new data to the patterns obser

In [15]:
import pandas as pd
import numpy as np
import re

# Define the extract_label function
def extract_label(llm_response):
    """Extract the predicted label from the LLM response."""
    match = re.search(r'(?i)label: (\d+)', llm_response)  # The '(?i)' makes the search case-insensitive
    if match:
        return int(match.group(1))
    return None

# Sample indexes from your training set
indexes = [0, 5, 32, 38, 58, 65]

# Function to get a random subset of row data
def get_random_subset(row, subset_size=10):
    """Returns a random subset of 'subset_size' elements from a row."""
    return row.sample(n=subset_size, random_state=1)

# Prepare few-shot training examples using a small subset of each row
few_shot_examples = []
for index in indexes:
    # Select a random subset of the row
    subset_data = get_random_subset(X_train.iloc[index])
    data_str = subset_data.to_string()
    actual_label = y_train.iloc[index].values[0]
    few_shot_examples.append(f"Data: {data_str}\nLabel: {actual_label}")

# Concatenate the few-shot examples for the prompt
few_shot_prompt = "\n\n".join(few_shot_examples)

# Lists to store results
llm_predictions = []
decision_tree_predictions = []
actual_labels = []

# Test the model on new data
test_indexes = [10, 15, 40, 100]  # Sample indexes from your test set
for index in test_indexes:
    # Select a random subset of the test row
    subset_data = get_random_subset(X_test.iloc[index])
    data_str = subset_data.to_string()
    query = (f"{few_shot_prompt}\n\n"
             f"Classify the following accelerometer data into the correct human activity.\n\n"
             f"Data: {data_str}\n\n"
             "Please provide the classification in the following format:\n"
             "Label: <predicted_label>")
    
    # Initialize and use the language model
    model_name = "llama3-70b"  # Model name
    llm = ChatGroq(model=groq_models[model_name], api_key=Groq_token, temperature=0)
    answer = llm.invoke(query)
    
    # Extract the LLM prediction
    llm_label = extract_label(answer.content.strip())
    
    # Store LLM prediction and actual label
    llm_predictions.append(llm_label)
    actual_labels.append(y_test.iloc[index].values[0])
    
    # Decision Tree prediction
    decision_tree_prediction = decision_tree_model.predict([X_test.iloc[index]])[0]
    decision_tree_predictions.append(decision_tree_prediction)

# Convert lists to numpy arrays for easier calculation
llm_predictions = np.array(llm_predictions)
decision_tree_predictions = np.array(decision_tree_predictions)
actual_labels = np.array(actual_labels)

# Calculate accuracy
def calculate_accuracy(predictions, actuals):
    """Calculate the accuracy of predictions compared to actual labels."""
    correct_predictions = np.sum(predictions == actuals)
    accuracy = correct_predictions / len(actuals)
    return accuracy

# Calculate accuracies
llm_accuracy = calculate_accuracy(llm_predictions, actual_labels)
decision_tree_accuracy = calculate_accuracy(decision_tree_predictions, actual_labels)

# Print results
print(f"Few-Shot Learning Accuracy: {llm_accuracy:.2f}")
print(f"Decision Tree Accuracy: {decision_tree_accuracy:.2f}")


NameError: name 'decision_tree_model' is not defined